In [3]:
import sys

sys.path.append("../model")

from Experiment import Experiment
from samplers.NFSampler import NFSampler
from samplers.SpikeSlabSampler import SpikeSlabSampler
from samplers.RandomWalkSampler import RandomWalkSampler
from searchers.GNNExplainerSearcher import GNNExplainerSearcher
from searchers.GreedySearcher import GreedySearcher

from torch_geometric.nn.models.explainer import clear_masks, set_masks

experiment = Experiment("syn3-full-verified", "..")
experiment.train_base_model()

Loading syn3 dataset


2022-03-09 10:02:43.984429: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /gpfs/runtime/opt/julia/1.7.1/lib:/gpfs/runtime/opt/gcc/8.3/lib64:/gpfs/runtime/opt/python/3.7.4/lib:/gpfs/runtime/opt/R/3.4.3_mkl/lib64/R/lib:/gpfs/runtime/opt/intel/2017.0/lib/intel64:/gpfs/runtime/opt/intel/2017.0/mkl/lib/intel64:/gpfs/runtime/opt/java/8u111/jre/lib/amd64
2022-03-09 10:02:43.984461: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
import torch
import pyro
from pyro import distributions as dist
from torch_geometric.utils import k_hop_subgraph
from pyro.infer import infer_discrete, config_enumerate


N = 565
subset, edge_index_adj, mapping, edge_mask_hard = k_hop_subgraph(
            N, 3, experiment.edge_index, relabel_nodes=True)

total_mapping = {k: i for k, i in enumerate(subset.tolist())}
x_adj = experiment.x[subset]

N = edge_index_adj.size(1)

with torch.no_grad():
    y = experiment.model(x_adj, edge_index_adj).exp()[mapping[0]]

def sample_model(data):
    f1 = pyro.sample("f1", dist.Beta(2, 10).expand([N]).to_event(1))
    set_masks(experiment.model, torch.nn.Parameter(f1), edge_index_adj, False)
    mean = experiment.model(data, edge_index_adj)[mapping].reshape(-1).exp()
    clear_masks(experiment.model)
    y_sample = pyro.sample("y_sample", dist.Categorical(probs=y/y.sum()))
    return pyro.sample("y_hat", dist.Categorical(probs=mean/mean.sum()), obs=y_sample)

In [5]:
from pyro.infer.mcmc import NUTS
from pyro.infer.mcmc import MCMC

nuts_kernel = NUTS(sample_model, adapt_step_size=True)
mcmc = MCMC(nuts_kernel, num_samples=1000, warmup_steps=250)
mcmc.run(x_adj)

Sample: 100%|██████████| 1250/1250 [00:43, 29.02it/s, step size=4.86e-01, acc. prob=0.859]


In [6]:
samples = mcmc.get_samples()
samples['f1'].mean(0)

tensor([0.1682, 0.1638, 0.1650, 0.1672, 0.1650, 0.1633, 0.1714, 0.1713, 0.1644,
        0.1701, 0.1646, 0.1579, 0.1712, 0.1665, 0.1680, 0.1698, 0.1711, 0.1661,
        0.1654, 0.1618, 0.1642, 0.1686, 0.1683, 0.1699, 0.1701, 0.1644, 0.1730,
        0.1656, 0.1640, 0.1684, 0.1651, 0.1651])